In [3]:
import pandas as pd
import mod_sql

In [4]:
## 클래스 선언

_db = mod_sql.Database()

### emp table
> 사원 정보를 담고 있는 테이블

### dept table
> 부서 정보를 담고 있는 테이블

In [5]:
sql = """ 
SELECT * from dept
"""

_db.executeAll(sql)

,DEPTNO,DNAME,LOC
0,10,ACCOUNTING,NEW YORK
1,20,RESEARCH,DALLAS
2,30,SALES,CHICAGO
3,40,OPERATIONS,BOSTON


### 부서명이 SALES 소속인 사원들을 출력
1. dept 테이블에서 부서이름이 sales인 DEPTNO을 가지고 온다.
2. emp 테이블에서 DEPTNO 1번 과정에서 나온 결과값을 기준으로 사원 정보 출력

In [6]:
sql = """ 
SELECT * from emp WHERE DEPTNO = (
    SELECT DEPTNO from dept WHERE DNAME = 'SALES'
)
"""

_db.executeAll(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.00,30
1,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.00,30
2,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.00,30
3,7698,BLAKE,MANAGER,7839,1981-05-01,2850.0,,30
4,7844,TURNER,SALESMAN,7698,1981-09-08,1500.0,0.00,30
5,7900,JAMES,CLERK,7698,1981-12-03,950.0,,30


### 부서번호가 10, 30인 사원 정보를 출력

In [7]:
sql = """ 
SELECT * from emp where DEPTNO = 10 or DEPTNO = 30
"""

sql_2 = """ 
SELECT * from emp WHERE DEPTNO in (10, 30)
"""


_db.executeAll(sql_2)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.00,30
1,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.00,30
2,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.00,30
3,7698,BLAKE,MANAGER,7839,1981-05-01,2850.0,,30
4,7782,CLARK,MANAGER,7839,1981-06-09,2450.0,,10
5,7839,KING,PRESIDENT,0,1981-11-17,5000.0,,10
6,7844,TURNER,SALESMAN,7698,1981-09-08,1500.0,0.00,30
7,7900,JAMES,CLERK,7698,1981-12-03,950.0,,30
8,7934,MILLER,CLERK,7782,1982-01-23,1300.0,,10


### job_code 새로운 파생변수 생성
1. salesman -> 1, manager -> 2, 그 외의 직업은 3 
2. EMPNO, ENAME, JOB, job_code 출력

In [8]:
sql = """ 
SELECT EMPNO, ENAME, JOB,
    CASE
        WHEN JOB = "SALESMAN"
        THEN "1"
        WHEN JOB = "manager"
        THEN "2"
        ELSE "3"
    END AS JOB_CODE
FROM emp
"""

_db.executeAll(sql)

,EMPNO,ENAME,JOB,JOB_CODE
0,7369,SMITH,CLERK,3
1,7499,ALLEN,SALESMAN,1
2,7521,WARD,SALESMAN,1
3,7566,JONES,MANAGER,2
4,7654,MARTIN,SALESMAN,1
5,7698,BLAKE,MANAGER,2
6,7782,CLARK,MANAGER,2
7,7788,SCOTT,ANALYST,3
8,7839,KING,PRESIDENT,3
9,7844,TURNER,SALESMAN,1


### 위에서 SQL구문으로 파생변수를 생성하였는데 pandas를 이용하여 같은 값을 출력해보시오.

In [9]:
sql = """ 
SELECt * from emp
"""

_emp = _db.executeAll(sql)

_emp.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902,1980-12-17,800.0,,20
1,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.00,30
2,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.00,30
3,7566,JONES,MANAGER,7839,1981-04-02,2975.0,,20
4,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.00,30


In [10]:
def job_code(x):
    if x == "SALESMAN":
        return '1'
    elif x == "MANAGER":
        return '2'
    else:
        return '3'    

In [11]:
# case1

_emp["JOB_CODE"] = _emp["JOB"].apply(job_code)

In [12]:
# case2

_emp["JOB_CODE"] = _emp["JOB_CODE"].apply(lambda x : '1' if (x == "SALESMAN")
                                          else '2' if (x == "MANAGER")
                                          else '3')

In [13]:
_emp.loc[:,["EMPNO", "ENAME", "JOB", "JOB_CODE"]].head()

,EMPNO,ENAME,JOB,JOB_CODE
0,7369,SMITH,CLERK,3
1,7499,ALLEN,SALESMAN,3
2,7521,WARD,SALESMAN,3
3,7566,JONES,MANAGER,3
4,7654,MARTIN,SALESMAN,3


In [14]:
_emp[["EMPNO", "ENAME", "JOB", "JOB_CODE"]].head()

,EMPNO,ENAME,JOB,JOB_CODE
0,7369,SMITH,CLERK,3
1,7499,ALLEN,SALESMAN,3
2,7521,WARD,SALESMAN,3
3,7566,JONES,MANAGER,3
4,7654,MARTIN,SALESMAN,3


In [15]:
## concat -> 결합
sql = """ 
SELECT concat(ENAME, '(', JOB, ')') as ENAME from emp
"""

_db.executeAll(sql)

,ENAME
0,SMITH(CLERK)
1,ALLEN(SALESMAN)
2,WARD(SALESMAN)
3,JONES(MANAGER)
4,MARTIN(SALESMAN)
5,BLAKE(MANAGER)
6,CLARK(MANAGER)
7,SCOTT(ANALYST)
8,KING(PRESIDENT)
9,TURNER(SALESMAN)


In [16]:
_emp["ENAME_"] = _emp["ENAME"] + '(' + _emp["JOB"] + ')'
_emp["ENAME_"]

0         SMITH(CLERK)
1      ALLEN(SALESMAN)
2       WARD(SALESMAN)
3       JONES(MANAGER)
4     MARTIN(SALESMAN)
5       BLAKE(MANAGER)
6       CLARK(MANAGER)
7       SCOTT(ANALYST)
8      KING(PRESIDENT)
9     TURNER(SALESMAN)
10        ADAMS(CLERK)
11        JAMES(CLERK)
12       FORD(ANALYST)
13       MILLER(CLERK)
14           test(CEO)
15           test(CEO)
16           test(CEO)
Name: ENAME_, dtype: object

In [17]:
## if문 사용한 sql 쿼리문
sql = """ 
SELECT if (SAL >= 2000, "HIGH", "LOW") as `H/L` from emp
"""

_db.executeAll(sql)

,H/L
0,LOW
1,LOW
2,LOW
3,HIGH
4,LOW
5,HIGH
6,HIGH
7,HIGH
8,HIGH
9,LOW


In [18]:
### pandas를 이용해서 위 sql문과 같은 작업을 실행
sql = """ 
SELECt * from emp
"""

_emp = _db.executeAll(sql)


In [19]:
_emp["H/L"] = _emp["SAL"].apply(lambda x : "HIGH" if (x >=2000) else "LOW")

In [20]:
_emp[["SAL", "H/L"]]

,SAL,H/L
0,800.0,LOW
1,1600.0,LOW
2,1250.0,LOW
3,2975.0,HIGH
4,1250.0,LOW
5,2850.0,HIGH
6,2450.0,HIGH
7,3000.0,HIGH
8,5000.0,HIGH
9,1500.0,LOW


In [21]:
sql = """ 
SELECT replace(JOB, "SALESMAN", "SALES") FROM emp
"""

_db.executeAll(sql)

,"replace(JOB, ""SALESMAN"", ""SALES"")"
0,CLERK
1,SALES
2,SALES
3,MANAGER
4,SALES
5,MANAGER
6,MANAGER
7,ANALYST
8,PRESIDENT
9,SALES


In [22]:
sql = """ 
SELECt * from emp
"""

_emp = _db.executeAll(sql)

In [23]:
_emp.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902,1980-12-17,800.0,,20
1,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.00,30
2,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.00,30
3,7566,JONES,MANAGER,7839,1981-04-02,2975.0,,20
4,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.00,30


In [24]:
_emp["JOB"] = _emp["JOB"].str.replace("SALESMAN", "SALES")

_emp["JOB"]

0         CLERK
1         SALES
2         SALES
3       MANAGER
4         SALES
5       MANAGER
6       MANAGER
7       ANALYST
8     PRESIDENT
9         SALES
10        CLERK
11        CLERK
12      ANALYST
13        CLERK
14          CEO
15          CEO
16          CEO
Name: JOB, dtype: object

In [25]:
### join -> left join
sql == """ 
SELECT * from emp left join dept on emp.DEPTNO = dept.DEPTNO
"""

_db.executeAll(sql).head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902,1980-12-17,800.0,,20
1,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.00,30
2,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.00,30
3,7566,JONES,MANAGER,7839,1981-04-02,2975.0,,20
4,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.00,30


In [26]:
sql = """ 
SELECT * from emp right join dept on emp.DEPTNO = dept.DEPTNO
"""

_db.executeAll(sql).head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7934.0,MILLER,CLERK,7782.0,1982-01-23,1300.0,,10.0,10,ACCOUNTING,NEW YORK
1,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,,10.0,10,ACCOUNTING,NEW YORK
2,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,,10.0,10,ACCOUNTING,NEW YORK
3,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,,20.0,20,RESEARCH,DALLAS
4,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,,20.0,20,RESEARCH,DALLAS


In [27]:
sql = """
INSERT into emp values (%s, %s, %s, %s, %s, %s, %s, %s)
"""

values = ["8000", "test", "CEO", 0, "1988-02-01", 3500.0, 100, 50] 

_db.execute(sql, values)


In [28]:
_db.commit()

In [ ]:
### inner join으로 데이터 결합
### 결과값이 어떻네 나오는지 확인

sql = """ 
SELECT * from emp inner join dept on emp.DEPTNO = dept.DEPTNO
"""

_db.executeAll(sql)

In [30]:
sql = """ 
SELECt * from emp
"""

sql_2 = """ 
SELECt * from dept
"""

_emp = _db.executeAll(sql)
_dept = _db.executeAll(sql_2)

In [31]:
emp_dept_left = pd.merge(_emp, _dept, on="DEPTNO", how = "left")

emp_dept_left.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,DNAME,LOC
0,7369,SMITH,CLERK,7902,1980-12-17,800.0,,20,RESEARCH,DALLAS
1,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.00,30,SALES,CHICAGO
2,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.00,30,SALES,CHICAGO
3,7566,JONES,MANAGER,7839,1981-04-02,2975.0,,20,RESEARCH,DALLAS
4,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.00,30,SALES,CHICAGO


In [32]:
emp_dept_right = pd.merge(_emp, _dept, on="DEPTNO", how = "right")

emp_dept_right.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,DNAME,LOC
0,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,,10,ACCOUNTING,NEW YORK
1,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,,10,ACCOUNTING,NEW YORK
2,7934.0,MILLER,CLERK,7782.0,1982-01-23,1300.0,,10,ACCOUNTING,NEW YORK
3,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,,20,RESEARCH,DALLAS
4,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,,20,RESEARCH,DALLAS


In [33]:
emp_dept_inner = pd.merge(_emp, _dept, on="DEPTNO", how = "inner")

emp_dept_inner.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,DNAME,LOC
0,7369,SMITH,CLERK,7902,1980-12-17,800.0,,20,RESEARCH,DALLAS
1,7566,JONES,MANAGER,7839,1981-04-02,2975.0,,20,RESEARCH,DALLAS
2,7788,SCOTT,ANALYST,7566,1987-06-28,3000.0,,20,RESEARCH,DALLAS
3,7876,ADAMS,CLERK,7788,1987-07-13,1100.0,,20,RESEARCH,DALLAS
4,7902,FORD,ANALYST,7566,1981-12-03,3000.0,,20,RESEARCH,DALLAS


In [34]:
emp_dept_outer = pd.merge(_emp, _dept, on="DEPTNO", how = "outer")

emp_dept_outer.tail()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,DNAME,LOC
14,8000.0,test,CEO,0.0,1988-02-01,3500.0,100,50,NaN,NaN
15,8000.0,test,CEO,0.0,1988-02-01,3500.0,100,50,NaN,NaN
16,8000.0,test,CEO,0.0,1988-02-01,3500.0,100,50,NaN,NaN
17,8000.0,test,CEO,0.0,1988-02-01,3500.0,100,50,NaN,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,OPERATIONS,BOSTON


In [35]:
sql = """ 
SELECT * FROM emp
WHERE SAL >= 2000 and SAL <= 5000
"""

_db.executeAll(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7566,JONES,MANAGER,7839,1981-04-02,2975.0,,20
1,7698,BLAKE,MANAGER,7839,1981-05-01,2850.0,,30
2,7782,CLARK,MANAGER,7839,1981-06-09,2450.0,,10
3,7788,SCOTT,ANALYST,7566,1987-06-28,3000.0,,20
4,7839,KING,PRESIDENT,0,1981-11-17,5000.0,,10
5,7902,FORD,ANALYST,7566,1981-12-03,3000.0,,20
6,8000,test,CEO,0,1988-02-01,3500.0,100,50
7,8000,test,CEO,0,1988-02-01,3500.0,100,50
8,8000,test,CEO,0,1988-02-01,3500.0,100,50
9,8000,test,CEO,0,1988-02-01,3500.0,100,50


In [36]:
sql = """ 
SELECt * FROM emp
WHERE SAL between 2000 and 5000
"""

_db.executeAll(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7566,JONES,MANAGER,7839,1981-04-02,2975.0,,20
1,7698,BLAKE,MANAGER,7839,1981-05-01,2850.0,,30
2,7782,CLARK,MANAGER,7839,1981-06-09,2450.0,,10
3,7788,SCOTT,ANALYST,7566,1987-06-28,3000.0,,20
4,7839,KING,PRESIDENT,0,1981-11-17,5000.0,,10
5,7902,FORD,ANALYST,7566,1981-12-03,3000.0,,20
6,8000,test,CEO,0,1988-02-01,3500.0,100,50
7,8000,test,CEO,0,1988-02-01,3500.0,100,50
8,8000,test,CEO,0,1988-02-01,3500.0,100,50
9,8000,test,CEO,0,1988-02-01,3500.0,100,50
